# Practical 4 - Worksheet

Written by Dr Joshua Kaggie, for the Data Intensive Science MPhil
Copyright Feb 2024

Portions of this have been adapted from
* https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
* https://github.com/rajatguptakgp/image_enhancement_techniques

Note the licence for the data in Module 1: Crystal Clean dataset, which is CC0 public domain. The data is available here: https://www.kaggle.com/datasets/mohammadhossein77/brain-tumors-dataset and licence information here: https://creativecommons.org/publicdomain/zero/1.0/

The licence for the data in Module 2 is for educational purposes only, and may not be redistributed outside of this course. The images come from the FERARI musculoskeletal study at Addenbrooke's Hospital: https://classic.clinicaltrials.gov/ct2/show/NCT03657303

In [1]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,10)

# Module 1 - Classification

Module 1 is adapted from example given by Sasank Chilamkurthy, licence BSD
https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

### Task  1: Run the following code. 

This may take some time, as you may need to figure out how to set up these environments. Try to figure out what you can on your own, but also use us to help debug. 

In [2]:
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.backends import cudnn

import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

In [3]:
cudnn.benchmark = True
plt.ion()

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(120),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(160),
        transforms.CenterCrop(120),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

## Download data to work with by running the following:

In [ ]:
!wget "https://www.dropbox.com/scl/fi/topy5v9m2krhtuy129yz8/glioma.zip?rlkey=mefwsowwndwvldz630qdkcj77&dl=0" -O glioma.zip

### Unzip it by running the following:

In [ ]:
!mkdir data
!mkdir data/train
!mkdir data/val
!mv *.zip data/
!unzip data/glioma.zip -d data/train/
!unzip data/glioma.zip -d data/val/

In [8]:
data_dir = 'data/'

In [9]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(120),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(160),
        transforms.CenterCrop(120),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [11]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model



In [14]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [15]:
model_ft = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### Task  2: Train the model! 

Train the model for 4 epochs. If you have time, try training the model for a longer period.

### Task  3: Visualise the model.

What function did we just programme?

### Task  4: Plot the change over each epoc.

Modify the programme so that you can track the change in accuracy over each epoch.

### Task  5: Instead of resnet16, use resnet50.

The following code may help:

from torchvision.models import resnet50, ResNet50_Weights

# Module 2 - PCA denoising

### Task  1: Run this code, get this data.

In [14]:
import scipy.io as sio
import glob
import sklearn
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Obtain the data using:

!wget "https://www.dropbox.com/scl/fi/f9xwxc4osoyqz823uyjhp/kneemri.mat?rlkey=f7jds6fyxmvn6i3z5sdquh3jo&dl=0" -O data/kneemri.mat 

In [84]:
kneemri = sio.loadmat('data/kneemri.mat')

### Task  2: Visualise the data.

How do you determine what is in the dataset?

### Task  3: PCA denoising

Use pca.fit on the image series to create a trained model. You will have to flatten the 2D dimension, meaning, reshape the 2D image dimensions into a 1D image dimension (so a matrix of 82 images x N^2, where N is the matrix size).  

Use the PCA denoising created from all images to denoise the first image in the series. This will use pca.transform and then pca.inverse_transform to a denoised image. Show the image.

Try with different a different number of PCA components.

Start with this code:

In [ ]:
from sklearn.decomposition import PCA, KernelPCA
pca = PCA(n_components=15, random_state=42)

### Task  4: PCA denoising, over many images

Try PCA denoising with 40 PCA components.

### Task  5: PCA denoising, over a single image

Try PCA denoising with 15 PCA components. But instead of using multiple images, perform this over a single image, using the features determined from the separate lines of the image.

### Task  5: Kernel PCA denoising, over many images

Use the following code, and perform PCA denoising over many images.

In [107]:
from sklearn.decomposition import KernelPCA

kernel_pca = KernelPCA(
    n_components=40,
    kernel="rbf",
    gamma=1e-4,
    fit_inverse_transform=True,
    alpha=5e-3,
    random_state=42,
)

### Extra: Can you use a Fourier filter to denoise? 

### Extra: Can you create a classifier to differentiate between images are 'sagittal', 'coronal', or 'axial'? These are the three directions used.